<a href="https://colab.research.google.com/github/pranath/spatial_analysis/blob/main/applications/8_flood_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple flood inundation model

Flooding is one of the most common and devastating natural disasters, which affects nearly every population on the globe. Geospatial models are a powerful tool in estimating the impact of a flood and mitigating that impact before it happens. We often hear on the news that a river is reaching the flood stage, but that information is meaningless if we can't understand the impact.

In this project we create a simple flood inundation model to help predict the extend of floods accross a specific terrian.

I have used the QGIS desktop application to render images of the different terrains.

# 1 Import libraries & define flood fill algorithm

The flood fill algorithm takes a starting point column and row (i.e. x,y co-ordinate) as well as an image mask of all areas below a certain height. It will then crawl from this point, in all 4 directions, accumulating all other points connected to this one that are also below the specified height, which will be the flood prediction returned as an array of points.

In [23]:
"""
Crawls a terrain raster from a starting
point and "floods" everything at the same
or lower elevation by producing a mask
image of 1, 0 values.
"""

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!cp -r /content/gdrive/My\ Drive/Datasets/FloodFill/* .

import numpy as np
import matplotlib.pyplot as plt
from linecache import getline

def floodFill(c, r, mask):
    """
    Crawls a mask array containing
    only 1 and 0 values from the
    starting point (c=column,
    r=row - a.k.a. x, y) and returns
    an array with all 1 values
    connected to the starting cell.
    This algorithm performs a 4-way
    check non-recursively.
    """
    # cells already filled
    filled = set()
    # cells to fill
    fill = set()
    fill.add((c, r))
    width = mask.shape[1]-1
    height = mask.shape[0]-1
    # Our output inundation array
    flood = np.zeros_like(mask, dtype=np.int8)
    # Loop through and modify the cells which
    # need to be checked.
    while fill:
        # Grab a cell
        x, y = fill.pop()
        if y == height or x == width or x < 0 or y < 0:
            # Don't fill
            continue
        if mask[y][x] == 1:
            # Do fill
            flood[y][x] = 1
            filled.add((x, y))
            # Check neighbors for 1 values
            west = (x-1, y)
            east = (x+1, y)
            north = (x, y-1)
            south = (x, y+1)
            if west not in filled:
                fill.add(west)
            if east not in filled:
                fill.add(east)
            if north not in filled:
                fill.add(north)
            if south not in filled:
                fill.add(south)
    return flood

Mounted at /content/gdrive


# 2 Open terrain data file

The terrain file is an ASCII grid data file, that contains terrain elevation data in a region of Texas, USA. Our starting point for the flood prediction, we will choose as somewhere close to the city of Houston - this is where we can see the postion of the red star on the terrain render. The elevation data is height in meters.

In [24]:
source = "terrain.asc"
target = "flood90.asc"

# Starting point for the
# flood inundation near Houston, Texas
sx = 2582
sy = 2057

print("Opening image...")
img = np.loadtxt(source, skiprows=6)
print("Image opened")

img

Opening image...
Image opened


array([[146., 144., 143., ..., 101.,  98.,  97.],
       [145., 144., 144., ...,  98.,  98.,  97.],
       [144., 144., 145., ...,  98.,  98.,  98.],
       ...,
       [ 94.,  95.,  95., ...,  63.,  63.,  64.],
       [ 93.,  94.,  94., ...,  62.,  61.,  61.],
       [ 89.,  90.,  91., ...,  61.,  60.,  59.]])

![title](flood-terrain.png)

# 3 Create image mask for flood fill algorithm

For our flood fill algorithm, we need to extract a mask of all the terrain pixels below a certain height. We will create two masks, one for areas below 70 meters and one for areas below 90 meters.

In [25]:
# a = np.where(img < 70, 1, 0)
a = np.where(img < 90, 1, 0)
print("Image masked")

Image masked


# 4 Run flood fill algorithm on image mask

We now run the flood fill algorithm on both our 70 & 90 meter flood scenario masks.

In [20]:
# Parse the headr using a loop and
# the built-in linecache module
hdr = [getline(source, i) for i in range(1, 7)]
values = [float(h.split(" ")[-1].strip()) for h in hdr]
cols, rows, lx, ly, cell, nd = values
xres = cell
yres = cell * -1

print("Beginning flood fill")
fld = floodFill(sx, sy, a)
print("Finished Flood fill")

header = ""
for i in range(6):
    header += hdr[i]

print("Saving grid")
# Open the output file, add the hdr, save the array
with open(target, "wb") as f:
    f.write(bytes(header, 'UTF-8'))
    np.savetxt(f, fld, fmt="%1i")
print("Done!")

Beginning flood fill
Finished Flood fill
Saving grid
Done!


## 4.1 Flood prediction of areas below 70 meters

This rendering shows us the potential areas flooded below 70 meters from our starting point in Houston.

![title](flood70.png)

## 4.2 Flood prediction of areas below 90 meters

This rendering shows us the potential areas flooded below 90 meters from our starting point in Houston. We can see how much more of the terrain would be flooded in this scenario.

![title](flood90.png)

# 5 Conclusion

We have seen how this simple model can be used to generate flood predictions for different scenarios of elevation. Hydrological flood models are expensive to develop and can be very complex. These models are essential for engineers in building flood control systems. However, first responders and potential flood victims are only interested in the impact of an impending flood.

This model starts with a single point and floods an area with the maximum volume of water that a flood basin can hold at a particular flood stage. Usually, this analysis is a worst-case scenario. Hundreds of other factors go into calculating how much water will enter into a basin from a river-topping flood stage. But we can still learn a lot from this simple first-order model.